In [ ]:
import datetime
def main():
    service = get_calendar_service()

    calId = 'ryan.jeffery.young@gmail.com'

    # call APi
    now = datetime.datetime.utcnow().isoformat() + 'Z' # z for UTC time
    tmr = datetime.datetime.utcnow() + timedelta(days=2)
    tmr_utc = tmr.isoformat() + 'Z'

    body = {
           "timeMin": now,
           "timeMax": tmr_utc,
           "timeZone": "PST",
           "items": [{"id": calId}]
       }
    
    freebusy_result = service.freebusy().query(body=body).execute()

    freebusy = freebusy_result[u'calendars']

    busyList = freebusy[calId]['busy']


    busy_cal = {}
    for cal_name in freebusy:
      busy_cal.update(freebusy[cal_name])

    print((busyList[0])['start'])


## Authentication

In [7]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [12]:
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

CREDENTIALS_FILE = 'credentials.json'

In [13]:
creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
service = build('calendar', 'v3', credentials=creds)

## Call API


In [91]:
import datetime
from datetime import timedelta
from datetimerange import DateTimeRange   
from setup import get_calendar_service

In [21]:
calId = 'ryan.jeffery.young@gmail.com'

In [97]:
# get todays date
today = datetime.date.today()
tomorrow = today + timedelta(days=1)

# convert date into datetime object at start of day, which is 800 UTC
start = datetime.datetime(today.year, today.month, today.day, 8, 1, 0) # 8am UTC is 12am PST
# convert to ISO format then append 'Z' for UTC time
start_UTC = start.isoformat() + 'Z'

end = datetime.datetime(today.year, today.month, tomorrow.day, 8, 0, 0) #7:59am UTC is 11:59pm PST
end_UTC = end.isoformat() + 'Z'

body = {
           "timeMin": start_UTC,
           "timeMax": end_UTC,
           "timeZone": 'PST',
           "items": [{"id": calId}]
       }
freebusy_result = service.freebusy().query(body=body).execute() # returns a dictionary 

freebusy_result

{'kind': 'calendar#freeBusy',
 'timeMin': '2020-11-24T08:01:00.000Z',
 'timeMax': '2020-11-25T08:00:00.000Z',
 'calendars': {'ryan.jeffery.young@gmail.com': {'busy': [{'start': '2020-11-24T12:00:00-08:00',
     'end': '2020-11-24T16:00:00-08:00'},
    {'start': '2020-11-24T22:30:00-08:00',
     'end': '2020-11-25T00:00:00-08:00'}]}}}

In [112]:
# set time range as start of today to end of day
time_range = DateTimeRange(start_UTC, end_UTC)
# get current time, in UTC
now =  datetime.datetime.utcnow().isoformat() + '+00:00'
future = datetime.datetime(2020, 11, 26, 8, 0, 0).isoformat() + '+00:00'

if now in time_range:
    print('yuh')
else:
    print('nah')


yuh


In [89]:
busy_list = freebusy_result['calendars'][calId]['busy'] # returns a list of busy times (in UTC)
busy_list

[{'start': '2020-11-24T12:00:00-08:00', 'end': '2020-11-24T16:00:00-08:00'},
 {'start': '2020-11-24T22:30:00-08:00', 'end': '2020-11-24T23:59:00-08:00'}]

In [29]:
len(busy_list)

2

2020-11-24T00:00:00Z
2020-11-24T23:59:59Z
